In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

referance notebook -https://www.kaggle.com/soumya9977/learning-to-sea-underwater-img-enhancement-eda

In [ ]:
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import re
import os
import random
! pip install tensorflow_addons
import tensorflow_addons as tfa
from matplotlib import animation, rc
import tensorflow as tf
import seaborn as sns
import random
import shutil
import ast

from IPython.display import HTML
from base64 import b64encode
import subprocess
import warnings 
warnings.filterwarnings('ignore')

df=pd.read_csv("../input/tensorflow-great-barrier-reef/train.csv")
test=pd.read_csv("../input/tensorflow-great-barrier-reef/test.csv")
sample=pd.read_csv("../input/tensorflow-great-barrier-reef/example_sample_submission.csv")

In [ ]:
display(df.head(5) , test.head(5) , sample)

In [ ]:
df1=df.copy()

In [ ]:
def plot_img(img_arr):
  '''function take input as array and plot image'''
  plt.figure(figsize = (15 , 5))
  plt.imshow(img_arr)
  plt.show()


In [ ]:
count=df['video_id'].count()
print(f"we have total  {count} no of images in dataframe")

> Dataframe have  3 different videos images.
> 
> those video images can seperated by varable video_id

In [ ]:
for i , j in enumerate(df['video_id'].value_counts().sort_values()):
    print(f"video {i} have {j} no. of images")
    
########################################################################

plt.figure(figsize=(8,5))
sns.countplot(df['video_id'], color='#2196F3')

In [ ]:
annot=[]
for i  in df["annotations"]:
    if i=="[]": annot.append(1)
    else:annot.append(0)

annot_True = annot.count(1)
annot_False = annot.count(0)

print(f"we have total {annot_True} no. of images which contain COTS in it")
print(f"we have total {annot_False} no. of images which dont have  COTS in it")

df["bounding_box_or_not"] = annot

sns.countplot(x ='bounding_box_or_not', data = df)

In [ ]:
# converting string annotations into list
df['annotations'] =df['annotations'].apply(eval)
########################################################################

#counting number of bounding boxes in each img  and adding it to new variable no_of_boundingBox

no_of_BoundingBox=[]
for i in tqdm(df["annotations"]):
    no_of_BoundingBox.append(len(i))
df["no_of_BoundingBox"]=no_of_BoundingBox
#######################################################################

#plot histogram
sns.countplot(df["no_of_BoundingBox"])

In [ ]:
sns.countplot(df["no_of_BoundingBox"])

> *18000 images dont have bounding box in it*
> 
> *there are over 2500 images which have single bounding box in image*
> 
> *very few images have 2+ bounding boxes in image*

In [ ]:
# removing images which dont have COTS/bounding_boxes 
df=df[df["no_of_BoundingBox"]>=1]
df.reset_index(drop=True, inplace=True)
df.head(5)

#plotting no_of_BoundingBox
fig, ax = plt.subplots()
sns.countplot(data=df, x="no_of_BoundingBox" , ax=ax)
plt.show()

In [ ]:
# removing images which dont have COTS/bounding_boxes 
df=df[df["no_of_BoundingBox"]>=1]
df.reset_index(drop=True, inplace=True)
df.head(5)

#plotting no_of_BoundingBox
fig, ax = plt.subplots()
sns.countplot(data=df, x="no_of_BoundingBox" , ax=ax)
plt.show()

> 2500+ images have 2 bounding box in image
> 
> 900+ images have 3 bounding box in image

In [ ]:
for i in range(90 , 100 , 1):
    value = round(np.percentile(df["no_of_BoundingBox"], i) , 0)
    print(f" {i}th percentile of data  contain {value} no.of COTS in image")

> we have 50% of images which have 1 bounding box in image 
> 
> 20% images have 2 bounding boxes in image
> 
> we will go further now to check how many bounding boxes in  90 to 100 percentile


In [ ]:
for i in range(90 , 100 , 1):
    k=round(np.percentile(df["no_of_BoundingBox"], i) , 0)
    print(f"{i} percentile values have {k} no. of bounding boxes in image")

> 1% of image have 16 bounding box in image
> 
> 1% of image have 14 bounding box in image
> 
> 1% of image have 11 bounding box in image


In [ ]:
# adding image path to data frame
dir="../input/tensorflow-great-barrier-reef/train_images"
df['image_path'] =dir + "/video_" + df['video_id'].astype(str) + "/" + df['video_frame'].astype(str) + ".jpg"

In [ ]:
print(df['image_path'][0])
print(df['image_path'][1])
print(df['image_path'][2])

In [ ]:
# adding two new variable to data frame image height and img_width
img = mpimg.imread(df["image_path"][0]) 

df["img_width"] = img.shape[1]
df["img_height"] = img.shape[0]

In [ ]:
img = mpimg.imread(df["image_path"][19]) 
img=cv2.rectangle(img, (520 , 151), (598 , 213), (0,0,0), 2)
img=cv2.rectangle(img, (598 , 204), (656 , 236), (0,0,0), 2)
plot_img(img)

In [ ]:
df["annotations"][19]

In [ ]:
id=35
bb=df["annotations"][id][0]
bb2=df["annotations"][id][1]
########################################################################

#calculating x_min , y_min , x_max , y_max for bounding box 1 

x_min=bb.get("x")
y_min=bb.get("y")
x_max=bb.get("x")+bb.get("width")
y_max=bb.get("y")+bb.get("height")

########################################################################

#calculating x_min , y_min , x_max , y_max for bounding box 2

x_min2=bb2.get("x")
y_min2=bb2.get("y")
x_max2=bb2.get("x")+bb2.get("width")
y_max2=bb2.get("y")+bb2.get("height")
print(x_min , y_min , x_max , y_max)
print(x_min2 , y_min2 , x_max2 , y_max2)
########################################################################

img = mpimg.imread(df["image_path"][id]) 

# drawwing rectangle bounding box on image for bounding box 1 & 2

img=cv2.rectangle(img, (x_min , y_min), (x_max , y_max), (255,0,0), 2)
img=cv2.rectangle(img,  (x_min2 , y_min2), (x_max2 , y_max2), (255,0,0), 2)
plot_img(img)

In [ ]:
# converting dict annotation to [ x , y , h , w ]
bounding_box=[]
for annot in tqdm(df["annotations"]):
    bb=[]
    for i in annot:
        bb_cordinate=[]
        bb_cordinate.append(i.get("x"))
        bb_cordinate.append(i.get("y"))
        bb_cordinate.append(i.get("x")+i.get("width"))
        bb_cordinate.append(i.get("y")+i.get("height"))
        bb.append(bb_cordinate)
    bounding_box.append(bb)
df["Bounding_Box"]=bounding_box

In [ ]:
df["Bounding_Box"][30:40]

In [ ]:
df.tail(5)

In [ ]:
def draw_bounding_box(id , data , img_arr=False ):
    '''given function draw bounding box in image , 
    it take input as image id and it output image with bounding box  '''
    if img_arr is not False:
        img=img_arr
    else: img=mpimg.imread(data["image_path"][id]) 
    bb=data["Bounding_Box"][id]
    for i in bb:
        
        x_min=i[0]
        y_min=i[1]
        x_max=i[2]
        y_max=i[3]
        img=cv2.rectangle(img, (x_min , y_min), (x_max , y_max), (255,0,0), 2)
    return img

In [ ]:
id=35 
img=draw_bounding_box(id , df)
plot_img(img)

> visualizing image after drawing bounding box on image with the help of function draw_bounding_box

In [ ]:
# creating dataframe which just have 13 bounding boxes in each image
df_13=df[df["no_of_BoundingBox"]>=13]    
df_13.sort_values("no_of_BoundingBox").head(5)
df_13.reset_index(drop=True, inplace=True)
df.shape

**why we are creating seperate data frame for 13 boxes?**

> because what we notice is that as we go deep into sea , we cant visualize COTS properly because of blue color saturation is more . as we go deep we see more COTS in depth
> 
> 
> we need to clear image such a way  we can clearly visualize COTS in images / we need to remove water from image so we can clearly visualize COTS in image

In [ ]:
id=30
img=draw_bounding_box( id , df_13)
plot_img(img)

> above image have 13 COTS in it , but we cant visualize most of them by eye so we need to do feature eng

In [ ]:
def make_video(data ,start , end ,  video_id ,fun_name ,  fun=False):
    # partly borrowed from https://github.com/RobMulla/helmet-assignment/blob/main/helmet_assignment/video.py
    fps = 15 # don't know exact value
    width = 1280
    height = 720
    df=data[data["video_id"]==video_id][start : end]
    save_path = f'video{video_id}-{start}-{end}-{fun_name}.mp4'
    tmp_path = "tmp_" + save_path
    output_video = cv2.VideoWriter(tmp_path, cv2.VideoWriter_fourcc(*"MP4V"), fps, (width, height))
    
    video_df = df.query('video_id == @video_id')
    for i  in tqdm(df.index):
      img=draw_bounding_box(i , df)
      img=cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
      if fun!=False:
        img = fun(img)
      output_video.write(img)
    
    output_video.release()
    # Not all browsers support the codec, we will re-load the file at tmp_output_path
    # and convert to a codec that is more broadly readable using ffmpeg
    if os.path.exists(save_path):
        os.remove(save_path)
    subprocess.run(
        ["ffmpeg", "-i", tmp_path, "-crf", "18", "-preset", "veryfast", "-vcodec", "libx264", save_path]
    )
    os.remove(tmp_path)

def plot_img(img_arr):
  '''function take input as array and plot image'''
  plt.figure(figsize = (15 , 5))
  plt.imshow(img_arr)
  plt.show()


In [ ]:
fun_name = "none"
start=15
end=100
video_id=0
make_video(df ,start , end , video_id ,fun_name)


mp4 = open(f'./video{video_id}-{start}-{end}-{fun_name}.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=1000 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
# creating dataframe which just have 13 bounding boxes in each image
df_13=df[df["no_of_BoundingBox"]>=13]    
df_13.sort_values("no_of_BoundingBox").head(5)
df_13.reset_index(drop=True, inplace=True)

#creating video 
fun_name = "none"
start=0
end=100
video_id= 1
make_video(df_13 ,start , end , video_id ,fun_name)


mp4 = open(f'./video{video_id}-{start}-{end}-{fun_name}.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=1000 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
# below both fucntion are use to convert set of images into animation 
rc('animation', html='jshtml')

def create_animation(ims):
    fig = plt.figure(figsize=(9, 9))
    plt.axis('off')
    im = plt.imshow(ims[0])

    def animate_func(i):
        im.set_array(ims[i])
        return [im]

    return animation.FuncAnimation(fig, animate_func, frames = len(ims), interval = 1000//12)

###############################################################################################

def animate_images(start , no_of_img , data  ,function):
  images=[]
  for i in tqdm(range(start , start+no_of_img)):
    img=mpimg.imread(data["image_path"][i])
    img1=function(img)

    bb= data["Bounding_Box"][i]
    for i in bb:
      x_min=i[0]
      y_min=i[1]
      x_max=i[2]
      y_max=i[3]
      img1=cv2.rectangle(img1, (x_min , y_min), (x_max , y_max), (255,0,0), 2)
    images.append(img1)
  return create_animation(images)

In [ ]:
images=[]
for i in tqdm(range(25 , 60)):
    img=mpimg.imread(df["image_path"][i])

    bb=df["Bounding_Box"][i]
    for i in bb:
      x_min=i[0]
      y_min=i[1]
      x_max=i[2]
      y_max=i[3]
      img=cv2.rectangle(img, (x_min , y_min), (x_max , y_max), (255,0,0), 2)
    images.append(img)

create_animation(images)

> n above animation we can see COTS properly because camera is close to COTS and video captured not in too depth

In [ ]:
images=[]
for i in tqdm(range(25 , 60)):
    img=mpimg.imread(df_13["image_path"][i])

    bb=df_13["Bounding_Box"][i]
    for i in bb:
      x_min=i[0]
      y_min=i[1]
      x_max=i[2]
      y_max=i[3]
      img=cv2.rectangle(img, (x_min , y_min), (x_max , y_max), (255,0,0), 2)
    images.append(img)

create_animation(images)

> as we go in depth of sea we found more number of COTS in image but we cant properly visualize image

Learning to Sea: Underwater img Enhancement + EDA
https://www.kaggle.com/soumya9977/learning-to-sea-underwater-img-enhancement-eda

# HSV - Hue Saturation Value
* HSV color space: It stores color information in a cylindrical representation of RGB color points.
* It attempts to depict the colors as perceived by the human eye.
* Hue value varies from 0-179, Saturation value varies from 0-255 and Value value varies from 0-255.

# ***histogram equalization***
>  the pixel values are are between 0-255, but we will not find any pixel values which are exactly 0 or 255, 
> it does not have any image which is pure white or pure black.
> If we apply the histogram equalization then it will reduce the color depth.Currently the minimum pixel value is 52 and the highest is 255. After you apply histogram equalization, you will find the the min pixel value now got transformed to zero and the max got converted to 255

In [ ]:
id=45
img = mpimg.imread(df_13["image_path"][id])
print("Original image")
plot_img(img)
######################################################
print("transformed image")
img = mpimg.imread(df_13["image_path"][id])
#converting RGB image to HSV image 
img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
#value channnel 
value=img_hsv[: ,: ,2]
#applying histogram equalization on value channnel
img_hsv[:, :, 2] = cv2.equalizeHist(value)
#converting HSV to RGB
image_RGB = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)
plot_img(image_RGB)
#######################################################
print("transformed image+bounding_Box")
img=draw_bounding_box(id  ,df_13 , image_RGB)
plot_img(img)

In [ ]:
def hist_equalization(img):
    #converting RGB image to HSV image 
    img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    #value channnel
    value=img_hsv[: ,: ,2]
    #applying histogram equalization on value channnel
    img_hsv[:, :, 2] = cv2.equalizeHist(value)
    #converting HSV to RGB
    image_RGB = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2RGB)
    return image_RGB

####################################################################

def plot_original_transformed_img(sample ,function  ):
    for k in range(0 , sample):
       #random smaple
       i=random.choice(list(df_13.index))
       path=df_13["image_path"][i]
       plt.figure(figsize = (10 , 3))
       #reading original imaage 
       img = mpimg.imread(path)
       plt.subplot(1,2,1)
       plt.imshow(img)
       #transforming image
       img=function(img)
       plt.subplot(1,2,2)
       plt.imshow(img)
       plt.show()
    return

In [ ]:
plot_original_transformed_img(3 , hist_equalization )

In [ ]:
#creating video 
fun_name = hist_equalization
start=0
end=100
video_id= 1
make_video(df_13 ,start , end , video_id ,fun_name ,fun_name)


mp4 = open(f'./video{video_id}-{start}-{end}-{fun_name}.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=1000 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

> histogram equalization not giving good results ,  we need to try another technique to clearify images 

# **Histogram Equalizers on each channel**

In [ ]:
def RecoverHE(img):
  #converting image to numpy array
    img= np.array(img)
    #appying histogram equaalization on each channel
    for i in range(3):
        img[:, :, i] =  cv2.equalizeHist(img[:, :, i])
    return img

In [ ]:
plot_original_transformed_img(3 , RecoverHE)

In [ ]:
# animate_images(30 ,60 , df_13 ,RecoverHE )

In [ ]:
#creating video 
fun_name = RecoverHE
start=0
end=500
video_id=1
make_video(df_13 ,start , end , video_id ,fun_name ,fun_name)


mp4 = open(f'./video{video_id}-{start}-{end}-{fun_name}.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=1000 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

> after applying Histogram Equalizers on each channel we  see  good results , we can use this transformation 

> lets try some more transformation technique

# **CLAHE Histogram Equalization**
> CLAHE operates on small regions in the image, called tiles, rather than the entire image. The neighboring tiles are then combined using bilinear interpolation to remove the artificial boundaries. 
> This algorithm can be applied to improve the contrast of images.

In [ ]:
img = mpimg.imread(df_13["image_path"][0])
plt.figure(figsize = (15 , 5))
plt.imshow(img)
plt.show()
#converting RGB to HSV
hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
#extracting H S V values
h, s, v = hsv_img[:,:,0], hsv_img[:,:,1], hsv_img[:,:,2]
#clipLimit=Threshold for contrast limiting. , 
#tileGridSize =Size of grid for histogram equalization. Input image will be divided into equally sized rectangular tiles. 
clahe = cv2.createCLAHE(clipLimit = 20, tileGridSize = (30 , 30))
#applying clache on value
v = clahe.apply(v)
#stacking HSV
hsv_img = np.dstack((h,s,v))
#converting HSV image back to RGB
rgb_img = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2RGB)
#plot image
plt.figure(figsize = (15 , 5))
plt.imshow(rgb_img)
plt.show()

In [ ]:
def clahe_hsv(img):
    #converting RGB to HSV
    hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    #extracting H S V values
    h, s, v = hsv_img[:,:,0], hsv_img[:,:,1], hsv_img[:,:,2]
    #clipLimit=Threshold for contrast limiting. , 
    #tileGridSize =Size of grid for histogram equalization. Input image will be divided into equally sized rectangular tiles.
    clahe = cv2.createCLAHE(clipLimit = 15, tileGridSize = (10 , 10))
    #applying clache on value
    v = clahe.apply(v)
    #stacking HSV
    hsv_img = np.dstack((h,s,v))
    #converting HSV image back to RGB
    rgb_img = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2RGB)
    return rgb_img

In [ ]:
plot_original_transformed_img(3 ,clahe_hsv )

In [ ]:
#creating video 
fun_name = clahe_hsv
start=0
end=500
video_id=2
make_video(df_13 ,start , end , video_id ,fun_name ,fun_name)


mp4 = open(f'./video{video_id}-{start}-{end}-{fun_name}.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=1000 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
# animate_images(30 ,60 , df_13 ,clahe_hsv)

> CLAHE Histogram Equalization is not giving me good result so i will skip this method

# RecoverCLAHE

In [ ]:
def RecoverCLAHE(img):
    clahe = cv2.createCLAHE(clipLimit=5, tileGridSize=(15 , 15))
    img= np.array(img)
    for i in range(3):

        img[:, :, i] = clahe.apply(img[:, :, i])

    return img

In [ ]:
plot_original_transformed_img(2 ,RecoverCLAHE )

In [ ]:
# animate_images(30 ,50 , df_13 ,RecoverCLAHE)

In [ ]:
#creating video 
fun_name = RecoverCLAHE
start=0
end=500
video_id=2
make_video(df_13 ,start , end , video_id ,fun_name ,fun_name)


mp4 = open(f'./video{video_id}-{start}-{end}-{fun_name}.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=1000 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

we cant use RecoverCLAHE  because its gving worst result

# tfa.image.equalize

In [ ]:
def tfimageEqualizer(img):
    img=tfa.image.equalize(img)
    return img

In [ ]:
images=[]
for i in tqdm(range(10  ,30+80)):
    img=mpimg.imread(df_13["image_path"][i])
    img=tfa.image.equalize(img)
    images.append(img)
    
final_images=[]   
for k in tqdm(range(0 , len(images))):
    img=np.array(images[k])
    m=10+k
    bb=df_13["Bounding_Box"][m]
    for i in bb:
       x_min=i[0]
       y_min=i[1]
       x_max=i[2]
       y_max=i[3]
       img=cv2.rectangle(img, (x_min , y_min), (x_max , y_max), (255,0,0), 2) 
    final_images.append(img)

In [ ]:
# create_animation(final_images)

In [ ]:
images=[]
for i in tqdm(range(10  ,30+80)):
    img=mpimg.imread(df_13["image_path"][i])
    img=tfa.image.equalize(img)
    images.append(img)
    
###############################################################################################     
final_images=[]   
for k in tqdm(range(0 , len(images))):
    img=np.array(images[k])
    m=10+k
    bb=df_13["Bounding_Box"][m]
    for i in bb:
       x_min=i[0]
       y_min=i[1]
       x_max=i[2]
       y_max=i[3]
       img=cv2.rectangle(img, (x_min , y_min), (x_max , y_max), (255,0,0), 2) 
    final_images.append(img)
############################################################################################### 
#creating video from Final Image list

fps = 15 # don't know exact value
width = 1280
height = 720
save_path = f'video_tfaEqualizer.mp4'
tmp_path = "tmp_" + save_path
output_video = cv2.VideoWriter(tmp_path, cv2.VideoWriter_fourcc(*"MP4V"), fps, (width, height))

for img  in tqdm(final_images):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    output_video.write(img)
output_video.release()

if os.path.exists(save_path):
        os.remove(save_path)
subprocess.run(
        ["ffmpeg", "-i", tmp_path, "-crf", "18", "-preset", "veryfast", "-vcodec", "libx264", save_path]
    )
os.remove(tmp_path)

############################################################################################### 
mp4 = open(f'./video_tfaEqualizer.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=1000 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

> tfa.image.equalize give us good result 
> 
> we can properly visualize COTS in image 
> 
> so we will use tfa.img.equalize as transformation